In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

# a = llm.predict("How many planets?")
b = chat.predict("How many planets?")

b

'There are currently eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [5]:
from langchain.schema import SystemMessage, AIMessage, HumanMessage

messages = [
    SystemMessage(
        content="You are a geography expert. You only reply in Korean."
    ),
    AIMessage(content="My name is JaeHun"),
    HumanMessage(content="Can I ask your name? What is distance btwn Mexico and Chille?")
]

chat.predict_messages(messages)

AIMessage(content='안녕하세요! 제 이름은 재훈입니다. 멕시코와 칠레 사이의 거리는 약 7,500 킬로미터입니다.')

In [7]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

template = PromptTemplate.from_template("What is distance btwn {country_a} and {country_b}?")

prompt = template.format(country_a="Thailand", country_b="France")

chat.predict(prompt)

'The distance between Thailand and France is approximately 9,927 kilometers (6,161 miles).'

In [9]:
template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "You are a geography expert. You only reply in {language}."
        ),
        AIMessagePromptTemplate.from_template("My name is {name}"),
        HumanMessagePromptTemplate.from_template("What's your name? What is distance btwn {country_a} and {country_b}?"),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mixico",
    country_b="Canada",
)

chat.predict_messages(prompt)

AIMessage(content='Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και του Καναδά είναι περίπου 4.500 χιλιόμετρα.')

In [10]:
messages = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. You only reply in {language}."),
        ("ai", "My name is {name}"),
        ("human", "What's your name? What is distance btwn {country_a} and {country_b}?"),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mixico",
    country_b="Canada",
)

chat.predict_messages(prompt)

AIMessage(content='Το όνομά μου είναι Socrates. Η απόσταση μεταξύ Μεξικού και Καναδά είναι περίπου 4.600 χιλιόμετρα.')

In [14]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse("apple, orange, grape")


['apple', 'orange', 'grape']

In [19]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="list planets."
)
result = chat.predict_messages(prompt)
p.parse(result.content)

['mercury', 'venus', 'earth', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune']

In [20]:
chain = template | chat | p
chain.invoke({
    "max_items": 5,
    "question": "list planets."
})

['mercury', 'venus', 'earth', 'mars', 'jupiter']